In [3]:
#!pip install -r requirements.txt

In [2]:
from torch_complex.tensor import ComplexTensor

In [4]:
import numpy as np


np.random.seed(2)

spect = np.random.rand(11, 5)
# spect = np.uniform()
print(spect.shape, spect)

spect = np.log1p(spect)

x = spect
# x = spect.T

mean = np.mean(x, axis=1, keepdims=True)
std = np.std(x, axis=1, ddof=1, keepdims=True)
print("std : ", std.shape, std)
std = np.clip(std, a_min=1e-20, a_max=None)
print("mean : ", mean.shape, mean)
print("std : ", std.shape, std)

x -= mean
x = x / std
print("x : ", x.shape, x)

# x = x.T
# print("x : ", x.shape, x)

(11, 5) [[0.4359949  0.02592623 0.54966248 0.43532239 0.4203678 ]
 [0.33033482 0.20464863 0.61927097 0.29965467 0.26682728]
 [0.62113383 0.52914209 0.13457995 0.51357812 0.18443987]
 [0.78533515 0.85397529 0.49423684 0.84656149 0.07964548]
 [0.50524609 0.0652865  0.42812233 0.09653092 0.12715997]
 [0.59674531 0.226012   0.10694568 0.22030621 0.34982629]
 [0.46778748 0.20174323 0.64040673 0.48306984 0.50523672]
 [0.38689265 0.79363745 0.58000418 0.1622986  0.70075235]
 [0.96455108 0.50000836 0.88952006 0.34161365 0.56714413]
 [0.42754596 0.43674726 0.77655918 0.53560417 0.95374223]
 [0.54420816 0.08209492 0.3663424  0.8508505  0.40627504]]
std :  (11, 1) [[0.16144169]
 [0.11323727]
 [0.16331676]
 [0.23071654]
 [0.1616835 ]
 [0.1384343 ]
 [0.11459825]
 [0.17431257]
 [0.16045515]
 [0.13854918]
 [0.19526545]]
mean :  (11, 1) [[0.30755925]
 [0.29044176]
 [0.32356832]
 [0.45770237]
 [0.20808267]
 [0.25448326]
 [0.37310758]
 [0.41004184]
 [0.49203707]
 [0.47833751]
 [0.35642744]]
std :  (11, 

In [7]:
import numpy as np


np.random.seed(2)

spect = np.random.rand(11, 5)
# spect = np.uniform()
print(spect.shape, spect)

# spect = np.log1p(spect)
# print(spect.shape, spect)

(11, 5) [[0.4359949  0.02592623 0.54966248 0.43532239 0.4203678 ]
 [0.33033482 0.20464863 0.61927097 0.29965467 0.26682728]
 [0.62113383 0.52914209 0.13457995 0.51357812 0.18443987]
 [0.78533515 0.85397529 0.49423684 0.84656149 0.07964548]
 [0.50524609 0.0652865  0.42812233 0.09653092 0.12715997]
 [0.59674531 0.226012   0.10694568 0.22030621 0.34982629]
 [0.46778748 0.20174323 0.64040673 0.48306984 0.50523672]
 [0.38689265 0.79363745 0.58000418 0.1622986  0.70075235]
 [0.96455108 0.50000836 0.88952006 0.34161365 0.56714413]
 [0.42754596 0.43674726 0.77655918 0.53560417 0.95374223]
 [0.54420816 0.08209492 0.3663424  0.8508505  0.40627504]]


In [11]:
import torch

y = spect

stft_conf = dict(
    n_fft = 320,
    win_length = 320,
    hop_length = 160,
    center = True,
    window = torch.hamming_window(window_length=320,
                                  dtype=torch.float32,
                                  device=torch.device('cpu')),
    normalized = False,
    onesided = True
)

y = torch.from_numpy(y)
print(y.size())
D = torch.stft(y, **stft_conf) # D.shape = (Freq, Frames, 2)
D = D.transpose(0, 1) # D.shape = (Frames, Freq, 2)

# 2. STFT --> Power Spectrum
input_stft = D
input_stft = ComplexTensor(input_stft[..., 0], input_stft[..., 1])

input_power = (input_stft.real ** 2) + (input_stft.imag ** 2)

torch.Size([11, 5])


RuntimeError: Argument #4: Padding size should be less than the corresponding input dimension, but got: padding (160, 160) at dimension 2 of input [1, 11, 5]

In [15]:
from torch import nn

outputs_channel = 32

seq = nn.Sequential(
            nn.Conv2d(1, outputs_channel, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5)),
            nn.BatchNorm2d(outputs_channel),
            nn.Hardtanh(0, 20, inplace=True),
            nn.Conv2d(outputs_channel, outputs_channel, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5)),
            nn.BatchNorm2d(outputs_channel),
            nn.Hardtanh(0, 20, inplace=True)
        )

In [18]:
for i, module in enumerate(seq):
    print(i, module)

0 Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
1 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
2 Hardtanh(min_val=0, max_val=20, inplace=True)
3 Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
4 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
5 Hardtanh(min_val=0, max_val=20, inplace=True)


In [17]:
print(seq.modules)

<bound method Module.modules of Sequential(
  (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Hardtanh(min_val=0, max_val=20, inplace=True)
  (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
  (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): Hardtanh(min_val=0, max_val=20, inplace=True)
)>


In [1]:
import copy

import numpy as np
import random

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

random_seed = 0

# torch.manual_seed(random_seed)
# torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
# np.random.seed(random_seed)
# random.seed(random_seed)

torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

B = 2
T = 6
V = 10
E = 2
H = 4


embedding_layer = nn.Embedding(V, E)
lstm = nn.LSTM(E, H, 1, batch_first=True)
print(lstm)

# lstm2 = nn.LSTM(E, H, 1)
# lstm2 = lstm2.load_state_dict(lstm.state_dict())
# print(lstm2)

# lstm2 = copy.deepcopy(lstm)
# lstm2.batch_first = False
# print(lstm2)

# lstm2 = nn.LSTM(E, H, 1, batch_first=False)
# print(lstm2)


for name_1 in lstm.named_parameters():
    print(name_1)


x = torch.LongTensor([[3, 9, 2, 1, 0, 0],
                      [1, 2, 4, 1, 1, 2]])

input_lengths = torch.LongTensor([4, 6])

embed = embedding_layer(x)



packed = pack_padded_sequence(embed, input_lengths, batch_first=True, enforce_sorted=False)
# print("pack:", packed)
# print(packed.data.size())

output, (h, c) = lstm(packed)
print(lstm.batch_first)
print("lstm: ", output)
print("h   : ", h.size())

output, output_lengths = pad_packed_sequence(output, batch_first=True)
print("pad : ", output.size(), output)

# print("#"*100)

# packed = pack_padded_sequence(embed, input_lengths, batch_first=True, enforce_sorted=False)
# #print("pack:", packed)
# # print(packed.data.size())

# output, (h, c) = lstm2(packed)
# print(lstm2.batch_first)
# print("lstm: ", output)
# print("h   : ", h.size())

# output, output_lengths = pad_packed_sequence(output, batch_first=True)
# print("pad : ", output.size(), output)






LSTM(2, 4, batch_first=True)
('weight_ih_l0', Parameter containing:
tensor([[-0.2919,  0.4298],
        [ 0.2231,  0.2423],
        [ 0.0263, -0.2563],
        [ 0.0846, -0.4668],
        [-0.3613, -0.2578],
        [ 0.3155,  0.2932],
        [-0.2217, -0.0180],
        [ 0.3198,  0.4971],
        [ 0.1984,  0.0675],
        [ 0.3352, -0.2944],
        [ 0.0932, -0.3877],
        [-0.3465, -0.2583],
        [ 0.2262,  0.2011],
        [-0.2962,  0.1511],
        [ 0.2745, -0.0631],
        [ 0.0191,  0.1159]], requires_grad=True))
('weight_hh_l0', Parameter containing:
tensor([[ 0.3102,  0.4801, -0.3853, -0.1832],
        [ 0.1965,  0.4143,  0.4351,  0.4412],
        [ 0.0995, -0.4348,  0.0460, -0.3128],
        [-0.4660,  0.4442,  0.3802, -0.4988],
        [ 0.0936, -0.0842, -0.0823, -0.2289],
        [ 0.1923, -0.2962,  0.1833,  0.2529],
        [ 0.3579,  0.1870, -0.4949, -0.3243],
        [ 0.2497,  0.1047, -0.3900, -0.2879],
        [ 0.4704,  0.3369, -0.2180, -0.1258],
        [

In [29]:
import copy

import numpy as np
import random

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

random_seed = 0

# torch.manual_seed(random_seed)
# torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
# np.random.seed(random_seed)
# random.seed(random_seed)

torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

B = 2
T = 6
V = 10
E = 2
H = 4


embedding_layer = nn.Embedding(V, E)
lstm = nn.LSTM(E, H, 1, batch_first=False)
print(lstm)

# lstm2 = nn.LSTM(E, H, 1)
# lstm2 = lstm2.load_state_dict(lstm.state_dict())
# print(lstm2)

# lstm2 = copy.deepcopy(lstm)
# lstm2.batch_first = False
# print(lstm2)

# lstm2 = nn.LSTM(E, H, 1, batch_first=False)
# print(lstm2)


for name_1 in lstm.named_parameters():
    print(name_1)

# for name_1, name_2 in zip(lstm.named_parameters(), lstm2.named_parameters()):
#     print(name_1, name_2)


x = torch.LongTensor([[3, 9, 2, 1, 0, 0],
                      [1, 2, 4, 1, 1, 2]])

input_lengths = torch.LongTensor([4, 6])

embed = embedding_layer(x)



# packed = pack_padded_sequence(embed, input_lengths, batch_first=True, enforce_sorted=False)
# # print("pack:", packed)
# # print(packed.data.size())

# output, (h, c) = lstm(packed)
# print(lstm.batch_first)
# print("lstm: ", output)
# print("h   : ", h.size())

# output, output_lengths = pad_packed_sequence(output, batch_first=True)
# print("pad : ", output.size(), output)

# print("#"*100)

packed = pack_padded_sequence(embed, input_lengths, batch_first=True, enforce_sorted=False)
#print("pack:", packed)
# print(packed.data.size())

output, (h, c) = lstm(packed)
print(lstm.batch_first)
print("lstm: ", output)
print("h   : ", h.size())

output, output_lengths = pad_packed_sequence(output, batch_first=True)
print("pad : ", output.size(), output)
print("output_lengths : ", output_lengths)






LSTM(2, 4)
('weight_ih_l0', Parameter containing:
tensor([[-0.2919,  0.4298],
        [ 0.2231,  0.2423],
        [ 0.0263, -0.2563],
        [ 0.0846, -0.4668],
        [-0.3613, -0.2578],
        [ 0.3155,  0.2932],
        [-0.2217, -0.0180],
        [ 0.3198,  0.4971],
        [ 0.1984,  0.0675],
        [ 0.3352, -0.2944],
        [ 0.0932, -0.3877],
        [-0.3465, -0.2583],
        [ 0.2262,  0.2011],
        [-0.2962,  0.1511],
        [ 0.2745, -0.0631],
        [ 0.0191,  0.1159]], requires_grad=True))
('weight_hh_l0', Parameter containing:
tensor([[ 0.3102,  0.4801, -0.3853, -0.1832],
        [ 0.1965,  0.4143,  0.4351,  0.4412],
        [ 0.0995, -0.4348,  0.0460, -0.3128],
        [-0.4660,  0.4442,  0.3802, -0.4988],
        [ 0.0936, -0.0842, -0.0823, -0.2289],
        [ 0.1923, -0.2962,  0.1833,  0.2529],
        [ 0.3579,  0.1870, -0.4949, -0.3243],
        [ 0.2497,  0.1047, -0.3900, -0.2879],
        [ 0.4704,  0.3369, -0.2180, -0.1258],
        [-0.4763, -0.0090, 

In [ ]:
h   :  torch.Size([1, 2, 4])
pad :  torch.Size([2, 6, 4]) tensor([[[ 0.1658, -0.1523, -0.0542, -0.0481],
         [ 0.1506,  0.0072,  0.1049, -0.1466],
         [ 0.0985,  0.0560,  0.2595, -0.0397],
         [ 0.1182,  0.0124,  0.2145, -0.0058],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0671, -0.0045,  0.1071, -0.0065],
         [ 0.0753,  0.0514,  0.2734, -0.0075],
         [ 0.1179,  0.0342,  0.2262, -0.0454],
         [ 0.1184, -0.0045,  0.2090, -0.0108],
         [ 0.1256, -0.0277,  0.2130, -0.0044],
         [ 0.1044,  0.0374,  0.3227, -0.0054]]], grad_fn=<IndexSelectBackward>)

In [4]:
x = torch.LongTensor([[3, 9, 2, 1, 0, 0],
                      [1, 2, 4, 1, 1, 2]])

x_np = x.cpu().numpy()

print(x_np.shape)

d = x_np[None]
print(d.shape)







(2, 6)
(1, 2, 6)


In [6]:
from collections import defaultdict

print(defaultdict)

<class 'collections.defaultdict'>


In [ ]:
from collections import defaultdict

sum_dict = defaultdict(lambda: 0)
sq_dict = defaultdict(lambda: 0)
count_dict = defaultdict(lambda: 0)



In [19]:
d = 0. + np.array([1, 2, 3, 4, 5])
print(d)
d = d + np.array([1, 2, 3, 4, 5])
print(d)

[1. 2. 3. 4. 5.]
[ 2.  4.  6.  8. 10.]


In [31]:


collect_stats = defaultdict(lambda: 0)

import numpy as np

np.random.seed(0)

# (Length, Dim) = (11, 5)
feat = np.random.rand(11, 5)
print(feat)

# print(len(feat))
# print(feat.sum(0))
# print((feat ** 2).sum(0))

collect_stats['count'] += len(feat)
collect_stats['sum'] += feat.sum(0)
collect_stats['sum_square'] += (feat ** 2).sum(0)

print(collect_stats['count'])
print(collect_stats['sum'])
print(collect_stats['sum_square'])

np.savez("feats_stats.npz",
         count=collect_stats['count'],
         sum=collect_stats['sum'],
         sum_square=collect_stats['sum_square'])


eps: float = 1.0e-20
stats_file = "feats_stats.npz"
stats = np.load(stats_file)

count = stats["count"]
sum_v = stats["sum"]
sum_square_v = stats["sum_square"]
mean = sum_v / count
var = sum_square_v / count - mean * mean
std = np.maximum(np.sqrt(var), eps)

print("#"*100)
print(count)
print(sum_v)
print(sum_square_v)


[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773   0.96366276 0.38344152]
 [0.79172504 0.52889492 0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985 0.77815675 0.87001215]
 [0.97861834 0.79915856 0.46147936 0.78052918 0.11827443]
 [0.63992102 0.14335329 0.94466892 0.52184832 0.41466194]
 [0.26455561 0.77423369 0.45615033 0.56843395 0.0187898 ]
 [0.6176355  0.61209572 0.616934   0.94374808 0.6818203 ]
 [0.3595079  0.43703195 0.6976312  0.06022547 0.66676672]
 [0.67063787 0.21038256 0.1289263  0.31542835 0.36371077]
 [0.57019677 0.43860151 0.98837384 0.10204481 0.20887676]]
11
[6.17463497 5.11674719 7.18936472 6.50455749 4.22104523]
[4.07557701 3.04404961 5.34875057 4.89666502 2.36013627]
####################################################################################################
11
[6.17463497 5.11674719 7.18936472 6.50455749 4.22104523]
[4.07557701 3.04404961 5.34875057 4.89666502 2.36013627]


In [37]:
a = np.array([1, 2, 3, 4, 5])

print(np.clip(a, a_min=2, a_max= 4))

print(np.maximum(a, 2))
print(np.clip(a, a_min=2, a_max=None))

[2 2 3 4 4]
[2 2 3 4 5]
[2 2 3 4 5]


In [ ]:
# utterance_mvn

(e2e.seq2seq) kwangho@ubuntu_gpu:~/lecture/e2e/seq2seq/ClovaCall$ cat log/LSTM_512x3_512x2_zeroth_korean_trimmed_utterance_mvn/run_las_asr_trainer_CUDA0.sh.log | grep Train
>> Train dataset :  data/zeroth_korean/train_zeroth_korean.trimmed.json
Train(train) Summary Epoch: [1] Average Loss 2.324      Average CER 92.940      Time 0:23:21.248011
Train(train) Summary Epoch: [2] Average Loss 1.898      Average CER 84.313      Time 0:48:26.438334
Train(train) Summary Epoch: [3] Average Loss 1.724      Average CER 80.214      Time 1:13:32.644493
Train(train) Summary Epoch: [4] Average Loss 1.593      Average CER 76.745      Time 1:38:44.240438
Train(train) Summary Epoch: [5] Average Loss 1.482      Average CER 73.035      Time 2:03:51.363183
Train(train) Summary Epoch: [6] Average Loss 1.384      Average CER 69.510      Time 2:28:59.029577
Train(train) Summary Epoch: [7] Average Loss 1.298      Average CER 65.870      Time 2:54:16.602752
Train(train) Summary Epoch: [8] Average Loss 1.223      Average CER 62.722      Time 3:19:35.680655
Train(train) Summary Epoch: [9] Average Loss 1.158      Average CER 59.833      Time 3:44:52.772413
(e2e.seq2seq) kwangho@ubuntu_gpu:~/lecture/e2e/seq2seq/ClovaCall$ cat log/LSTM_512x3_512x2_zeroth_korean_trimmed_utterance_mvn/run_las_asr_trainer_CUDA0.sh.log | grep Test
>> Test dataset :  ['data/zeroth_korean/test_zeroth_korean.trimmed.json']
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [1]     Average Loss 6.455      Average CER 156.792
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [2]     Average Loss 7.706      Average CER 164.226
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [3]     Average Loss 7.822      Average CER 146.557
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [4]     Average Loss 8.531      Average CER 169.235
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [5]     Average Loss 8.584      Average CER 154.148
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [6]     Average Loss 9.207      Average CER 140.357
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [7]     Average Loss 9.219      Average CER 111.287
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [8]     Average Loss 9.430      Average CER 111.816
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [9]     Average Loss 9.803      Average CER 105.195

In [ ]:
# instance

(base) kwangho@ubuntu_gpu:~/lecture/e2e/seq2seq/ClovaCall (master) $ cat log/LSTM_512x3_512x2_zeroth_korean_trimmed_instance/run_las_asr_trainer_CUDA1.sh.log
 | grep Train
>> Train dataset :  data/zeroth_korean/train_zeroth_korean.trimmed.json
Train(train) Summary Epoch: [1] Average Loss 2.325      Average CER 92.884      Time 0:22:57.441221
Train(train) Summary Epoch: [2] Average Loss 1.899      Average CER 84.224      Time 0:47:45.848677
Train(train) Summary Epoch: [3] Average Loss 1.726      Average CER 80.213      Time 1:12:33.175360
Train(train) Summary Epoch: [4] Average Loss 1.593      Average CER 76.660      Time 1:37:19.637668
Train(train) Summary Epoch: [5] Average Loss 1.481      Average CER 72.869      Time 2:01:58.303707
Train(train) Summary Epoch: [6] Average Loss 1.383      Average CER 69.265      Time 2:26:57.283479
Train(train) Summary Epoch: [7] Average Loss 1.294      Average CER 65.681      Time 2:51:57.533144
Train(train) Summary Epoch: [8] Average Loss 1.209      Average CER 62.061      Time 3:16:50.789678
Train(train) Summary Epoch: [9] Average Loss 1.136      Average CER 58.900      Time 3:41:37.104262
(base) kwangho@ubuntu_gpu:~/lecture/e2e/seq2seq/ClovaCall (master) $ cat log/LSTM_512x3_512x2_zeroth_korean_trimmed_instance/run_las_asr_trainer_CUDA1.sh.log
 | grep Test
>> Test dataset :  ['data/zeroth_korean/test_zeroth_korean.trimmed.json']
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [1]     Average Loss 6.504      Average CER 156.828
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [2]     Average Loss 8.054      Average CER 166.938
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [3]     Average Loss 7.635      Average CER 150.762
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [4]     Average Loss 8.356      Average CER 154.868
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [5]     Average Loss 8.642      Average CER 135.753
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [6]     Average Loss 9.127      Average CER 112.251
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [7]     Average Loss 9.395      Average CER 103.774
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [8]     Average Loss 9.421      Average CER 106.755
Test(data/zeroth_korean/test_zeroth_korean.trimmed.json) Summary Epoch: [9]     Average Loss 9.912      Average CER 102.592

In [27]:
import numpy as np
import torch

d = np.array([-np.inf])
print(d)
dt = torch.from_numpy(d).float()
print(dt)
print(np.exp(-np.inf))



d = torch.FloatTensor([1, 2, -float('inf')])
print(d, d.dtype)


r = torch.nn.Softmax(dim=-1)(d)
print(r)



[-inf]
tensor([-inf])
0.0
tensor([1., 2., -inf]) torch.float32
tensor([0.2689, 0.7311, 0.0000])


In [54]:
import torch
import numpy as np
import random


seed = 123456
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)


emb = torch.nn.Embedding(7, 10)
rnn = torch.nn.LSTM(10, 5, 2, batch_first=False, dropout=0.0, bidirectional=False)



seq = torch.tensor([[1,2,0], [3,0,0], [4,5,6], [3, 4, 0]])
lens = [2, 1, 3, 2]
print(seq)


_emb = emb(seq)
print("_emb: ", _emb.size())

packed = pack_padded_sequence(_emb, lens, batch_first=True, enforce_sorted=False)
print(packed)


out, hid = rnn(packed)
print("hid: ", hid[0].size())



seq_unpacked, lens_unpacked = pad_packed_sequence(packed, batch_first=True)
print(seq_unpacked.size(), seq_unpacked)
print(lens_unpacked)

tensor([[1, 2, 0],
        [3, 0, 0],
        [4, 5, 6],
        [3, 4, 0]])
_emb:  torch.Size([4, 3, 10])
PackedSequence(data=tensor([[ 1.1247, -0.1292,  1.9093, -0.5734,  0.3151,  0.7885,  0.0279,  0.3433,
          1.3762,  0.8962],
        [ 0.5047, -0.4987, -0.4891,  0.0578,  0.9436,  0.9775,  0.7843, -0.1034,
          0.1187, -0.2737],
        [ 1.0828, -1.7865, -0.7372, -0.7429, -0.4891,  2.4613, -0.7179, -0.2446,
          0.6929, -0.3115],
        [ 1.0828, -1.7865, -0.7372, -0.7429, -0.4891,  2.4613, -0.7179, -0.2446,
          0.6929, -0.3115],
        [ 1.2807, -1.8030, -0.4645, -0.6141, -0.6766, -0.3215, -1.0906, -0.7074,
          0.2326, -0.0978],
        [ 1.1537,  1.3910,  1.0732, -1.2210,  0.9103,  0.8257,  0.1247,  0.8079,
          0.1415, -0.5531],
        [ 1.1247, -0.1292,  1.9093, -0.5734,  0.3151,  0.7885,  0.0279,  0.3433,
          1.3762,  0.8962],
        [-0.2917,  0.6454,  2.2931, -0.5244,  1.2735, -3.2211,  0.1115,  0.1109,
         -0.7313, -0.3875]], 

In [ ]:
pad_packed_sequence
>>> seq = torch.tensor([[1,2,0], [3,0,0], [4,5,6]])
>>> lens = [2, 1, 3]
>>> packed = pack_padded_sequence(seq, lens, batch_first=True, enforce_sorted=False)
>>> packed
PackedSequence(data=tensor([4, 1, 3, 5, 2, 6]), batch_sizes=tensor([3, 2, 1]),
               sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))
>>> seq_unpacked, lens_unpacked = pad_packed_sequence(packed, batch_first=True)
>>> seq_unpacked
tensor([[1, 2, 0],
        [3, 0, 0],
        [4, 5, 6]])
>>> lens_unpacked
tensor([2, 1, 3])

In [74]:
import soundfile as sf
import torchaudio

y, sr = sf.read('./data/zeroth_korean/train_data_01/003/106/106_003_0077.trimmed.wav')

sig = torch.FloatTensor(y).unsqueeze(0)
feat = torchaudio.compliance.kaldi.fbank(sig,
                                         dither= 0.0,
                                         frame_length=25.0,
                                         frame_shift=10.0,
                                         sample_frequency=16000.0,
                                         num_mel_bins=80)
print(feat.size())




torch.Size([1147, 80])


In [4]:
import torch

L = 2
N = 3
A = torch.randn(L,L)


D_expand = A.expand(N, L, L) # specifies new size  # view
D_repeat = A.repeat(N,1,1) # specifies number of copies # copy

print(D_expand.size(), D_expand)
print(D_repeat.size(), D_repeat)

torch.Size([3, 2, 2]) tensor([[[0.1574, 0.8316],
         [0.4780, 0.8731]],

        [[0.1574, 0.8316],
         [0.4780, 0.8731]],

        [[0.1574, 0.8316],
         [0.4780, 0.8731]]])
torch.Size([3, 2, 2]) tensor([[[0.1574, 0.8316],
         [0.4780, 0.8731]],

        [[0.1574, 0.8316],
         [0.4780, 0.8731]],

        [[0.1574, 0.8316],
         [0.4780, 0.8731]]])


In [5]:
vars

<function vars>

In [6]:
help(vars)

Help on built-in function vars in module builtins:

vars(...)
    vars([object]) -> dictionary
    
    Without arguments, equivalent to locals().
    With an argument, equivalent to object.__dict__.



In [7]:
vars()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import torch\n\nL = 10\nN = 20\nA = torch.randn(L,L)\nA.expand(N, L, L) # specifies new size\nA.repeat(N,1,1) # specifies number of copies',
  'import torch\n\nL = 2\nN = 3\nA = torch.randn(L,L)\n\n\nA.expand(N, L, L) # specifies new size\nA.repeat(N,1,1) # specifies number of copies',
  'import torch\n\nL = 2\nN = 3\nA = torch.randn(L,L)\n\n\nD_expand = A.expand(N, L, L) # specifies new size\nD_repeat = A.repeat(N,1,1) # specifies number of copies\n\nprint(D_expand.size(), D_expand)',
  'import torch\n\nL = 2\nN = 3\nA = torch.randn(L,L)\n\n\nD_expand = A.expand(N, L, L) # specifies new size\nD_repeat = A.repeat(N,1,1) # specifies number of copies\n\nprint(D_expand.size(), D_expand)\nprint(D_repeat.size(), D_r